# Model Inspection

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

import seaborn as sns
sns.set_theme(font_scale=1.2)
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['savefig.bbox'] = 'tight'
plt.rcParams["savefig.dpi"] = 300

sklearn.set_config(display='diagram')

## Load the dataset

In [ ]:
from sklearn.datasets import fetch_california_housing

california = fetch_california_housing(as_frame=True)
X, y = california.data, california.target

In [ ]:
X.head()

In [ ]:
y.head()

### Insert random data for demonstration

In [ ]:
import numpy as np

X = X.assign(
    ran_num=np.arange(0, X.shape[0]))

### Split dataset

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42)

## Train linear model

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline

ridge = Pipeline([
    ('scale', StandardScaler()),
    ('reg', Ridge())
])
ridge.fit(X_train, y_train)

In [ ]:
ridge.score(X_train, y_train)

In [ ]:
ridge.score(X_test, y_test)

## Plot coefficients

Coefficients represent the relationship between a feature and the target assuming that all other features remain constant.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_linear_coef(coefs, names, ax=None, sorted=False):
    if ax is None:
        fig, ax = plt.subplots()
    coefs = pd.DataFrame(
       coefs, columns=['Coefficients'],
       index=names
    )
    
    if sorted:
        coefs = coefs.sort_values(by='Coefficients')

    coefs.plot(kind='barh', ax=ax)
    ax.axvline(x=0, color='.5')
    return ax

plot_linear_coef(ridge['reg'].coef_, names=X_train.columns, sorted=True);

## Coefficient variability

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedKFold

In [ ]:
cv_ridges = cross_validate(
    ridge, X, y, cv=RepeatedKFold(n_splits=5, n_repeats=5),
    return_estimator=True)

In [ ]:
cv_ridges

In [ ]:
coefs = pd.DataFrame(
   [model['reg'].coef_ for model in cv_ridges['estimator']],
   columns=X.columns
)

In [ ]:
coefs.head()

### Plotting the variability of the cofficients

In [ ]:
fig, ax = plt.subplots()
_ = plt.boxplot(coefs, vert=False, labels=coefs.columns)

## Exercise 1

1. Use a `Lasso` to fit the training dataset with `alpha=0.04`. **Hint:** Be sure to use a pipeline.
3. Plot `Lasso`'s coefficients next to the `Ridge` coefficients. How do they differ? **Hint** Use `plot_linear_coef`.
3. Use `RepeatedKFold` and `cross_validate` to check the variability of cofficients for `Lasso`.

In [ ]:
# %load solutions/03-ex01-solutions.py

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)

In [ ]:
rf.score(X_train, y_train)

In [ ]:
rf.score(X_test, y_test)

In [ ]:
def plot_importances(importances, names, ax=None):
    if ax is None:
        fig, ax = plt.subplots()
    indices = np.argsort(importances)
    ax.barh(range(len(importances)), importances[indices])
    ax.set(yticks=range(len(importances)),
           yticklabels=np.array(names)[indices]);

In [ ]:
importances = rf.feature_importances_
plot_importances(importances, X_train.columns);

Pay attention to `ran_num`!

## Permutation Feature Importance

### First on Test data

In [ ]:
from sklearn.inspection import permutation_importance

rf_result_test = permutation_importance(rf, X_test, y_test,
                                        n_repeats=15, n_jobs=-1)

In [ ]:
def plot_permutation_importance(perm_results, names, ax=None):
    perm_sorted_idx = perm_results.importances_mean.argsort()
    if ax is None:
        fig, ax = plt.subplots()
    _ = ax.boxplot(perm_results.importances[perm_sorted_idx].T, vert=False,
                   labels=np.array(names)[perm_sorted_idx])
    return ax

In [ ]:
_ = plot_permutation_importance(rf_result_test, X_test.columns)

## Exercise 2

1. Compute the permutation importance for `Lasso` on the test set.

In [ ]:
# %load solutions/03-ex02-solutions.py

### Load cancer dataset

In [ ]:
from sklearn.datasets import load_breast_cancer

data = load_breast_cancer()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

In [ ]:
rf.score(X_test, y_test)

### Permutation importance with random forest

In [ ]:
from sklearn.inspection import permutation_importance

rf_result = permutation_importance(rf, X_train, y_train,
                                   n_repeats=30, n_jobs=-1)

### Training data

In [ ]:
_ = plot_permutation_importance(rf_result, data.feature_names)

#### Most features are not useful?

In [ ]:
from scipy.stats import spearmanr
from scipy.cluster import hierarchy

corr = spearmanr(X_train).correlation
corr_linkage = hierarchy.ward(corr)

fig, ax = plt.subplots(figsize=(18, 12))
dendro = hierarchy.dendrogram(
    corr_linkage, labels=data.feature_names.tolist(),
    orientation='right', ax=ax)

In [ ]:
fig, ax = plt.subplots()
dendro_idx = np.arange(0, len(dendro['ivl']))

ax.imshow(corr[dendro['leaves'], :][:, dendro['leaves']], cmap='viridis')
ax.set_xticks(dendro_idx)
ax.set_yticks(dendro_idx)
ax.set_xticklabels(dendro['ivl'], rotation='vertical')
ax.set_yticklabels(dendro['ivl']);

Manutally pick a threshold based on visual inspection of the dendrogram to group faetures.

In [ ]:
from collections import defaultdict

cluster_ids = hierarchy.fcluster(corr_linkage, 1, criterion='distance')
cluster_id_to_feature_ids = defaultdict(list)
for idx, cluster_id in enumerate(cluster_ids):
    cluster_id_to_feature_ids[cluster_id].append(idx)
selected_features = [v[0] for v in cluster_id_to_feature_ids.values()]

In [ ]:
X_train_sel = X_train[:, selected_features]
X_test_sel = X_test[:, selected_features]

In [ ]:
rf_sel = RandomForestClassifier(random_state=42)
rf_sel.fit(X_train_sel, y_train)
print("Accuracy on test data with features removed: {:.2f}".format(
      rf_sel.score(X_test_sel, y_test)))

### Feature importance with selected features

In [ ]:
from sklearn.inspection import permutation_importance

rf_sel_result = permutation_importance(
    rf_sel, X_test_sel, y_test, n_repeats=30, n_jobs=-1)

In [ ]:
features_sel = data.feature_names[selected_features]
_ = plot_permutation_importance(rf_sel_result, features_sel)

## Partial Dependence

### Train a HistGradientBostingClassifer

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier 

In [ ]:
hist = HistGradientBoostingClassifier(random_state=0)
hist.fit(X_train_sel, y_train)

In [ ]:
hist.score(X_test_sel, y_test)

In [ ]:
from sklearn.inspection import plot_partial_dependence

In [ ]:
plot_partial_dependence(hist, X_train_sel,
                        features=['mean radius', 'mean concavity',
                                  'mean texture', 'mean symmetry'],
                        feature_names=features_sel,
                        n_cols=2)

## Exercise 3

1. Load the boston dataset using `sklearn.datasets.load_boston`.
1. Split the data into a training and test set.
1. Train a `sklearn.ensemble.GradientBoostingRegressor` on the training set and evalute on the test set.
1. Plot the feature_importances_ uses `plot_importances`
1. What are the 4 most important features according to `feature_importances_`?
1. What are the 4 most important features according to permutation importance on the test set?
1. Plot the partial dependence for the 4 most important features according to permutation importance.
1. Plot the partial dependence setting `features=[('LSTAT', 'RM')]` to get a bivariate parital dependence plot.

In [ ]:
# %load solutions/03-ex03-solutions.py